# Merge and Analyze Coinone USDT Monthly Candles

This notebook merges monthly CSVs into a single DataFrame, sorts chronologically, lets you exclude full KST dates from analysis, and computes median/average of the `close` column.


In [16]:
import os
import glob
import pandas as pd

pd.set_option("display.max_rows", 200)

# Adjust if needed
DATA_DIR = "../data/coinone/monthly/usdt"
PATTERN = "usdt_1m_*.csv.gz"  # expects gzip-compressed CSVs created by downloader

all_files = sorted(glob.glob(os.path.join(DATA_DIR, PATTERN)))
print(f"Found {len(all_files)} files")
for f in all_files[:]:
    print("-", os.path.basename(f))

Found 22 files
- usdt_1m_2023-11.csv.gz
- usdt_1m_2023-12.csv.gz
- usdt_1m_2024-01.csv.gz
- usdt_1m_2024-02.csv.gz
- usdt_1m_2024-03.csv.gz
- usdt_1m_2024-04.csv.gz
- usdt_1m_2024-05.csv.gz
- usdt_1m_2024-06.csv.gz
- usdt_1m_2024-07.csv.gz
- usdt_1m_2024-08.csv.gz
- usdt_1m_2024-09.csv.gz
- usdt_1m_2024-10.csv.gz
- usdt_1m_2024-11.csv.gz
- usdt_1m_2024-12.csv.gz
- usdt_1m_2025-01.csv.gz
- usdt_1m_2025-02.csv.gz
- usdt_1m_2025-03.csv.gz
- usdt_1m_2025-04.csv.gz
- usdt_1m_2025-05.csv.gz
- usdt_1m_2025-06.csv.gz
- usdt_1m_2025-07.csv.gz
- usdt_1m_2025-08.csv.gz


# merge all csv 

In [17]:
# Merge files into a single DataFrame, ensuring parsed UTC timestamps

if not all_files:
    raise SystemExit("No files found. Check DATA_DIR and PATTERN.")

# Read with parsing to ensure 'timestamp_utc' is datetime (as produced by downloader)
# Fallback: if reading generic, still parse when column exists
frames = []
for path in all_files:
    try:
        df = pd.read_csv(path, compression="gzip", parse_dates=["timestamp_utc"], dtype=str)
    except (pd.errors.ParserError, ValueError, OSError, UnicodeError):
        df = pd.read_csv(path, compression="gzip")
        if "timestamp_utc" in df.columns:
            df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], utc=True, errors="coerce")
    frames.append(df)

merged_df = pd.concat(frames, ignore_index=True)

# Sort by utc timestamp if available, otherwise try 'date' column
if "timestamp_utc" in merged_df.columns:
    merged_df = merged_df.drop_duplicates(subset=["timestamp_utc"]).sort_values("timestamp_utc").reset_index(drop=True)
elif "date" in merged_df.columns:
    merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")
    merged_df = merged_df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
else:
    raise SystemExit("Neither 'timestamp_utc' nor 'date' column found.")

print(merged_df.shape)
merged_df.head()


(908640, 11)


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst
0,2023-11-30 00:00:00,USDT/KRW,1m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 15:00:00+00:00,1701270000000,2023-11
1,2023-11-30 00:01:00,USDT/KRW,1m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 15:01:00+00:00,1701270060000,2023-11
2,2023-11-30 00:02:00,USDT/KRW,1m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 15:02:00+00:00,1701270120000,2023-11
3,2023-11-30 00:03:00,USDT/KRW,1m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 15:03:00+00:00,1701270180000,2023-11
4,2023-11-30 00:04:00,USDT/KRW,1m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 15:04:00+00:00,1701270240000,2023-11


# Convert UTC to KST date to enable full-day exclusion in KST


In [18]:

if "timestamp_utc" in merged_df.columns:
    # Ensure 'timestamp_utc' is a timezone-aware datetime
    if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):
        merged_df["timestamp_utc"] = pd.to_datetime(merged_df["timestamp_utc"], utc=True, errors="coerce")
    # Now safe to use .dt.tz_convert
    kst = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")
    merged_df["date_kst"] = kst.dt.date
else:
    # If only generic 'date' exists, assume it's timezone-naive local time; adapt as needed
    merged_df["date_kst"] = pd.to_datetime(merged_df["date"], errors="coerce").dt.date

merged_df[["timestamp_utc", "date_kst", "close"]].head()


/var/folders/4q/54t22rpj7ngdvjyv_g50m2dr0000gn/T/ipykernel_44885/2688033363.py:3: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):


,timestamp_utc,date_kst,close
0,2023-11-29 15:00:00+00:00,2023-11-30,1289.0
1,2023-11-29 15:01:00+00:00,2023-11-30,1289.0
2,2023-11-29 15:02:00+00:00,2023-11-30,1289.0
3,2023-11-29 15:03:00+00:00,2023-11-30,1289.0
4,2023-11-29 15:04:00+00:00,2023-11-30,1289.0


# Exclude specific data

In [ ]:
# Exclude by KST 1-hour windows and/or full KST dates
# - Windows: e.g., "2024-01-03_10:00" excludes [10:00, 11:00) on that day
# - Dates: e.g., "2024-02-13" excludes the entire day

# exclude_windows = ["2024-01-03_10:00"]

## note
# 2023-12-01_11:00, 2023-12-01_12:00 close 이상무
# 2023-12-14_12:00 close 이상무
# 2024-01-27_00:00 -> close는 아무 이상 없음

exclude_dates = ["2023-11-29", "2023-11-30"] # initial period
# exclude_windows = ["2024-04-14_05:00" "2024-03-06_04:00", "2024-04-22_15:00"]
exclude_windows = ["2024-04-14_05:00"]
# exclude_windows = []

In [20]:
# Parse KST datetime starts
raw_starts = pd.to_datetime(exclude_windows, format="%Y-%m-%d_%H:%M", errors="coerce")
raw_starts = raw_starts[~raw_starts.isna()]
if len(raw_starts) > 0:
    starts_kst = raw_starts.tz_localize("Asia/Seoul")
else:
    starts_kst = raw_starts

# Ensure required columns and types
if "timestamp_utc" not in merged_df.columns:
    raise SystemExit("Expected 'timestamp_utc' column for KST exclusion")

if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):
    merged_df["timestamp_utc"] = pd.to_datetime(merged_df["timestamp_utc"], utc=True, errors="coerce")
    merged_df = merged_df.dropna(subset=["timestamp_utc"]).reset_index(drop=True)

if "date_kst" not in merged_df.columns:
    merged_df["date_kst"] = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul").dt.date

kst_ts = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")

# 1) Window-based exclusion mask
mask_window = pd.Series(False, index=merged_df.index)
for start in starts_kst:
    end = start + pd.Timedelta(hours=1)
    mask_window |= (kst_ts >= start) & (kst_ts < end)

# 2) Full-day exclusion mask based on KST date
parsed_dates = pd.to_datetime(exclude_dates, errors="coerce") if exclude_dates else pd.to_datetime([])
if hasattr(parsed_dates, "isna"):
    parsed_dates = parsed_dates[~parsed_dates.isna()]
exclude_date_objs = set(pd.Series(parsed_dates).dt.date.tolist()) if len(parsed_dates) > 0 else set()
mask_day = merged_df["date_kst"].isin(exclude_date_objs) if len(exclude_date_objs) > 0 else pd.Series(False, index=merged_df.index)

# Combine masks
mask_excluded = mask_window | mask_day
filtered_df = merged_df.loc[~mask_excluded].copy()

print({
    "total_rows": len(merged_df),
    "excluded_rows": int(mask_excluded.sum()),
    "excluded_window_rows": int(mask_window.sum()),
    "excluded_day_rows": int(mask_day.sum()),
    "remaining_rows": len(filtered_df),
    "excluded_windows": [f"{s.strftime('%Y-%m-%d %H:%M')}~{(s + pd.Timedelta(hours=1)).strftime('%H:%M')} KST" for s in starts_kst],
    "excluded_dates": sorted([d.isoformat() for d in exclude_date_objs]),
})
filtered_df.head()

/var/folders/4q/54t22rpj7ngdvjyv_g50m2dr0000gn/T/ipykernel_44885/902665243.py:13: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):


{'total_rows': 908640, 'excluded_rows': 1440, 'excluded_window_rows': 0, 'excluded_day_rows': 1440, 'remaining_rows': 907200, 'excluded_windows': [], 'excluded_dates': ['2023-11-29', '2023-11-30']}


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst,date_kst
1440,2023-12-01 00:00:00,USDT/KRW,1m,1342.0,1342.0,1342.0,1342.0,0.0,2023-11-30 15:00:00+00:00,1701356400000,2023-12,2023-12-01
1441,2023-12-01 00:01:00,USDT/KRW,1m,1342.0,1342.0,1342.0,1342.0,0.0,2023-11-30 15:01:00+00:00,1701356460000,2023-12,2023-12-01
1442,2023-12-01 00:02:00,USDT/KRW,1m,1342.0,1342.0,1342.0,1342.0,0.0,2023-11-30 15:02:00+00:00,1701356520000,2023-12,2023-12-01
1443,2023-12-01 00:03:00,USDT/KRW,1m,1342.0,1342.0,1342.0,1342.0,0.0,2023-11-30 15:03:00+00:00,1701356580000,2023-12,2023-12-01
1444,2023-12-01 00:04:00,USDT/KRW,1m,1337.0,1337.0,1337.0,1337.0,7.9984,2023-11-30 15:04:00+00:00,1701356640000,2023-12,2023-12-01


# [debug] Filter by KST datetime range [start, end)

In [21]:

# Example: 2024-04-14_05:00 to 2024-04-14_06:00
range_start_kst_str = "2024-04-14_05:00"
range_end_kst_str = "2024-04-14_06:00"

if "timestamp_utc" not in filtered_df.columns:
    raise SystemExit("Expected 'timestamp_utc' in filtered_df")

# Parse inputs as KST-aware times
range_start_kst = pd.to_datetime(range_start_kst_str, format="%Y-%m-%d_%H:%M", errors="coerce").tz_localize("Asia/Seoul")
range_end_kst = pd.to_datetime(range_end_kst_str, format="%Y-%m-%d_%H:%M", errors="coerce").tz_localize("Asia/Seoul")

# Convert filtered_df timestamps to KST and filter
kst_ts = filtered_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")
mask_range = (kst_ts >= range_start_kst) & (kst_ts < range_end_kst)
window_df = filtered_df.loc[mask_range].copy()

print({
    "rows_in_window": len(window_df),
    "start_kst": range_start_kst.strftime("%Y-%m-%d %H:%M"),
    "end_kst": range_end_kst.strftime("%Y-%m-%d %H:%M"),
})
window_df.head(120)


{'rows_in_window': 60, 'start_kst': '2024-04-14 05:00', 'end_kst': '2024-04-14 06:00'}


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst,date_kst
196140,2024-04-14 05:00:00,USDT/KRW,1m,1537.0,1537.0,1523.0,1530.0,35698.10040386,2024-04-13 20:00:00+00:00,1713038400000,2024-04,2024-04-14
196141,2024-04-14 05:01:00,USDT/KRW,1m,1531.0,1531.0,1526.0,1530.0,36330.57358827,2024-04-13 20:01:00+00:00,1713038460000,2024-04,2024-04-14
196142,2024-04-14 05:02:00,USDT/KRW,1m,1530.0,1542.0,1524.0,1542.0,26935.49271257,2024-04-13 20:02:00+00:00,1713038520000,2024-04,2024-04-14
196143,2024-04-14 05:03:00,USDT/KRW,1m,1542.0,1550.0,1533.0,1550.0,15414.27219805,2024-04-13 20:03:00+00:00,1713038580000,2024-04,2024-04-14
196144,2024-04-14 05:04:00,USDT/KRW,1m,1549.0,1549.0,1532.0,1533.0,8424.48095498,2024-04-13 20:04:00+00:00,1713038640000,2024-04,2024-04-14
196145,2024-04-14 05:05:00,USDT/KRW,1m,1533.0,1540.0,1530.0,1537.0,4230.91091764,2024-04-13 20:05:00+00:00,1713038700000,2024-04,2024-04-14
196146,2024-04-14 05:06:00,USDT/KRW,1m,1537.0,1542.0,1533.0,1541.0,25436.05526397,2024-04-13 20:06:00+00:00,1713038760000,2024-04,2024-04-14
196147,2024-04-14 05:07:00,USDT/KRW,1m,1542.0,1587.0,1541.0,1575.0,9607.7218063,2024-04-13 20:07:00+00:00,1713038820000,2024-04,2024-04-14
196148,2024-04-14 05:08:00,USDT/KRW,1m,1588.0,1596.0,1575.0,1587.0,36580.41798744,2024-04-13 20:08:00+00:00,1713038880000,2024-04,2024-04-14
196149,2024-04-14 05:09:00,USDT/KRW,1m,1591.0,1618.0,1587.0,1616.0,30652.82654526,2024-04-13 20:09:00+00:00,1713038940000,2024-04,2024-04-14


In [22]:
# Compute statistics on 'close' after exclusion

# Ensure 'close' is numeric
for col in ("close",):
    if col in filtered_df.columns:
        filtered_df[col] = pd.to_numeric(filtered_df[col], errors="coerce")

median_close = filtered_df["close"].median()
average_close = filtered_df["close"].mean()

print(f"Median of close: {median_close}")
print(f"Average of close: {average_close}")


Median of close: 1399.0
Average of close: 1410.1801395502644
